In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

### Master Course Info
#### Double periods, Multiple Course Instances, Min, Max, HS/MS Category, Required Grades, Room Type

This file is constructed by the school

In [2]:
course_info = pd.read_excel('resources/Refactored_Files/Input1_Refactored.xlsx', delimiter=',')

#### Teacher Info

Using the first input file from the school, we constuct a file template for the school to input teacher/course matches 
This output file is "Teacher Template"

In [3]:
courses = pd.DataFrame(course_info['Course Name'], columns = ['Course Name', 'Teacher Name'])

# add multiple instances of courses to teacher doc
add = pd.DataFrame() 

# iterate over course with multiple instances
for index, row in course_info.loc[course_info['Number of Instances'] >= 1].iterrows():
    instances = np.arange(row['Number of Instances']) + 1
    for i in instances[1:]: # add a course for every additional instance
        c = row['Course Name'] + ' V'+ str(int(i))
        updated_row = row.copy()
        updated_row['Course Name'] = c
        add = add.append(updated_row, ignore_index=True)

courses = pd.concat([pd.DataFrame(add['Course Name'], columns = ['Course Name']), courses], axis=0).sort_values(by=['Course Name'])
courses.reset_index(drop=True).to_csv('resources/Teacher_Template.csv')

### Construct Input Files for LP Solver
#### Course Room Requirements, (Max, Min, MS Category, HS Category, Double, Teacher Info) 

Using the info from the school, construct input files that are readable by our LP program

In [6]:
all_info = pd.concat([course_info, add], axis=0).sort_values(by=['Course Name']).reset_index(drop=True)

# add in double periods 
add_double = pd.DataFrame() 
double = all_info.copy()
double.dropna(subset=['Double Period'])
for index, row in all_info.loc[all_info['Double Period'] == 'Yes'].iterrows():
    c = row['Course Name'] + ' II'
    updated_row = row.copy()
    updated_row['Course Name'] = c
    add_double = add_double.append(updated_row, ignore_index=True)    
add_double
all_info = pd.concat([all_info, add_double], axis=0).sort_values(by=['Course Name'])

# convert "yes" to 1
all_info['Double Period'] = all_info['Double Period'].map({'Yes': 1})

all_info.reset_index(drop=True).to_csv('resources/Refactored_Files/LP_Input.csv')
# all_info